# Process Data from 2015 into a consistent format.

> This notebook brings the 2015 into alignment with the desired format with respect to field name, type, and grouping.

In [ ]:
# Imports ----
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
import pickle

In [ ]:
from g2fd.internal import *

In [ ]:
#| default_exp internal

In [ ]:
# 2015
year_string = '2015'

          # print('Note! Many management factors are recorded in 2015!')
meta_path = './data/raw/G2F_Planting_Season_2015_v2/z._2015_supplemental_info/g2f_2015_field_metadata.csv' 
phno_path = './data/raw/G2F_Planting_Season_2015_v2/a._2015_hybrid_phenotypic_data/g2f_2015_hybrid_data_clean.csv' 
# geno_path = None,  
wthr_path = './data/raw/G2F_Planting_Season_2015_v2/b._2015_weather_data/g2f_2015_weather.csv'
soil_path = './data/raw/G2F_Planting_Season_2015_v2/d._2015_soil_data/g2f_2015_soil_data.csv'
          # There is data to be had but it's not formatted in a machine friendly way.
          # I've reformatted it to be easy to read in.
mgmt_path = './data/Manual_old/g2f_2015_agronomic information.csv'

meta = pd.read_csv(meta_path, encoding = "ISO-8859-1", low_memory=False)
phno = pd.read_csv(phno_path, encoding = "ISO-8859-1", low_memory=False)
wthr = pd.read_csv(wthr_path, encoding = "ISO-8859-1", low_memory=False)
soil = pd.read_csv(soil_path, encoding = "ISO-8859-1", low_memory=False)
mgmt = pd.read_csv(mgmt_path, encoding = "ISO-8859-1", low_memory=False)

In [ ]:
# load dicts for column renaming
meta_name_dict = mk_name_dict(name = 'meta')
phno_name_dict = mk_name_dict(name = 'phno')
soil_name_dict = mk_name_dict(name = 'soil')
wthr_name_dict = mk_name_dict(name = 'wthr')
mgmt_name_dict = mk_name_dict(name = 'mgmt')

# Rename
**Naming rules:**
- One dict for each input df
- Comment out anything that shouldn't be changed
- Upper_Upper_Unit_\$unit
- Upper_$number
- No special characters


In [ ]:
(find_unrecognized_columns(df = meta, dct = meta_name_dict),
find_unrecognized_columns(df = phno, dct = phno_name_dict),
find_unrecognized_columns(df = soil, dct = soil_name_dict),
find_unrecognized_columns(df = wthr, dct = wthr_name_dict),
find_unrecognized_columns(df = mgmt, dct = mgmt_name_dict))

([], [], [], [], [])

In [ ]:
meta = meta.rename(columns=meta_name_dict)
phno = phno.rename(columns=phno_name_dict)
soil = soil.rename(columns=soil_name_dict)
wthr = wthr.rename(columns=wthr_name_dict)
mgmt = mgmt.rename(columns=mgmt_name_dict)

# add indicator columns to help with debugging merge
meta['meta'] = True
phno['phno'] = True
soil['soil'] = True
wthr['wthr'] = True
mgmt['mgmt'] = True

In [ ]:
[e.shape for e in [meta, phno, soil, wthr, mgmt]]

[(54, 44), (13693, 39), (48, 12), (220044, 30), (157, 11)]

# Sanatize ID columns as needed


In [ ]:
simple_rename_dict = {
    'IA(H4)': 'IAH4', 
    'MN(?)1': 'MNu1', 
    'IA(?)3': 'IAu3', 
    'NY?': 'NYu', 
    'IA(?)2': 'IAu2'    
}

split_rename_dict = {
    'NEH1  NEH4': ['NEH1', 'NEH4'], 
    'KSH1  KSI1': ['KSH1', 'KSI1'], 
    'AZI1  AZI2': ['AZI1', 'AZI2'], 
    'PAI1  PAI2': ['PAI1', 'PAI2'], 
    'WIH1  WII1': ['WIH1', 'WII1'], 
    'WIH2  WII2': ['WIH2', 'WII2'], 
    'INH1  INI1': ['INH1', 'INI1'], 
    'NYH1  NYI1': ['NYH1', 'NYI1'], 
    'NYH3  NYI2': ['NYH3', 'NYI2'],  
    'ILH1  ILI1  ILH2': ['ILH1', 'ILI1', 'ILH2'],
    'TXH1  TXI1  TXI2': ['TXH1', 'TXI1', 'TXI2'],
    'MOH1  MOI1  MOH2  MOI2': ['MOH1', 'MOI1', 'MOH2', 'MOI2']
}

soil = sanitize_Experiment_Codes(
    df = soil, 
    simple_renames = simple_rename_dict, 
    split_renames = split_rename_dict)

wthr = sanitize_Experiment_Codes(
    df = wthr, 
    simple_renames = simple_rename_dict, 
    split_renames = split_rename_dict)

In [ ]:
# confirm everything's okay
print(
  'meta', find_unrecognized_experiments(meta.Experiment_Code, return_all_exps=False), 
'\nphno', find_unrecognized_experiments(phno.Experiment_Code, return_all_exps=False),
'\nsoil', find_unrecognized_experiments(soil.Experiment_Code, return_all_exps=False),
'\nwthr', find_unrecognized_experiments(wthr.Experiment_Code, return_all_exps=False),
'\nmgmt', find_unrecognized_experiments(mgmt.Experiment_Code, return_all_exps=False),
'\nall ', find_unrecognized_experiments([], return_all_exps=True)
)  

meta [] 
phno [] 
soil [] 
wthr [] 
mgmt [] 
all  ['ARH1', 'ARH2', 'AZH1', 'AZI1', 'AZI2', 'COH1', 'DEH1', 'DEI1', 'G2FDE1', 'G2FIA3', 'G2FIL1', 'G2FIN1', 'G2FMN2', 'G2FNE1', 'G2FNY1', 'G2FWI-HYB', 'G2FWI1', 'G2FWI2', 'G2F_IN_TX1', 'GA2', 'GAH1', 'GAH2', 'GAI1', 'GAI2', 'GEH1', 'GEH2', 'GXE_inb_BO2', 'GXE_inb_IA1', 'GXE_inb_IA2', 'GXE_inb_MO1', 'GXE_inb_MO3', 'GxE_inb_PA1', 'IAH1', 'IAH1a', 'IAH1b', 'IAH1c', 'IAH2', 'IAH2 ', 'IAH3', 'IAH3 ', 'IAH4', 'IAH4 ', 'IAI1', 'IAI2', 'IAI3', 'IAI4', 'IAu2', 'IAu3', 'ILH1', 'ILH2', 'ILI1', 'INH1', 'INI1', 'KSH1', 'KSH2', 'KSH3', 'KSI1', 'MIH1', 'MNH1', 'MNI1', 'MNI2', 'MNu1', 'MOH1', 'MOH1 ', 'MOH1-Rep1', 'MOH1-Rep2', 'MOH2', 'MOI1', 'MOI2', 'MOI3', 'NC1', 'NCH1', 'NCI1', 'NEH1', 'NEH2', 'NEH3', 'NEH4', 'NEI1', 'NYH1', 'NYH1', 'NYH2', 'NYH3', 'NYH4', 'NYI1', 'NYI2', 'NYS1', 'NYu', 'OHH1', 'ONH1', 'ONH2', 'PAI1', 'PAI2', 'SCH1', 'SDH1', 'SDI1', 'TX3', 'TXH1', 'TXH1-Dry', 'TXH1-Early', 'TXH1-Late', 'TXH2', 'TXH3', 'TXH4', 'TXI1', 'TXI2', 'TXI3', 'W

# Rearrange columns

## Process and reshape fertilizer data

In [ ]:
# Frustratingly `Fertilizer_Product` is only partially redundant. For some it contains information on amount 
# What I want to do is to keep `Fertilizer_Product` as `Product` but add an ingredient column to capture the npk data

# 1. create a single schema to parse
# 2. parse and merge into the desired table

temp = meta.loc[:, [ 'Experiment_Code',
 'N_Unit_lbs_per_A',
 'P_Unit_lbs_per_A',
 'K_Unit_lbs_per_A',
 'Fertilizer_Product',
 'Fertilizer_Application_Datetime_1',
 'Fertilizer_Application_Datetime_2',
 'Fertilizer_Application_Datetime_3',
 'Fertilizer_Application_Datetime_4',
 'Fertilizer_Application_Datetime_5',
 'Fertilizer_Application_Datetime_6',
 'Fertilizer_Application_Datetime_7',
 'Fertilizer_Application_Datetime_8'
                   ]]

for e in ['N_Unit_lbs_per_A',
 'P_Unit_lbs_per_A',
 'K_Unit_lbs_per_A',
 'Fertilizer_Product']:
    temp.loc[temp[e].isna(), e]=0
    temp[e] = temp[e].astype('string')
    
    
temp.loc[:, 'Fertilizer_info'] = temp.loc[:, 'N_Unit_lbs_per_A']+' (N) '+temp.loc[:, 'P_Unit_lbs_per_A']+' (P) '+temp.loc[:, 'K_Unit_lbs_per_A']+' (K) '+' ['+temp.loc[:, 'Fertilizer_Product']+']'
temp = temp.drop(columns= ['N_Unit_lbs_per_A',
                          'P_Unit_lbs_per_A',
                          'K_Unit_lbs_per_A',
                          'Fertilizer_Product'])

In [ ]:
# Not done here but this site has within field variation in management, so it would be best to resolve or drop it.
# PAI2
# '0 (N) 0 (P) 0 (K)  [urea; applied to south half  (Hi-N side) of field only]'
# PAI1
# '130 (N) 0 (P) 0 (K)  [urea (46-0-0); applied to south half (1/2 Acre, Hi-N side) of field only: actual applied was 140 lb urea to the Hi-N side of the field]'


# safe create imputation notes
temp = safe_create_col(temp, "Imputation_Notes")


mask = temp.Fertilizer_info == '234 (N) 98 (P) 51 (K)  [Pre Plant incorporated 162# DAP & 85# potash dry spread; Planter 2x2 liquid starter 25#N & 24#P; Sidedress 180#N liquid 28% ]'
temp.loc[mask, ['Fertilizer_info', 'Imputation_Notes']] = ['234 (N) 98 (P) 51 (K)', 'PrePlant']

mask = temp.Fertilizer_info == '120lbs/acre in UAN applied prior to spring tillage. (N) 0 (P) 80 (K)  [UAN, potash]'
temp = temp.loc[~mask, :]
temp = temp.merge(pd.DataFrame(
{'Experiment_Code': ['IAI3', 'IAI3'],
'Fertilizer_Application_Datetime_1': ['4/16/15', '4/22/15'],
'Fertilizer_info': ['120 (N)', '80 (K)']
}
), how = 'outer')

# '150 lbs/acre in UAN form applied with chemical ahead of tillage (N) 18-60-120-15-1.5 (n-p-k-sulfur-zinc) (P) 120lbs, fall applied (K)  [Uan, MESZ, and Potash]'
# unclear how much 18-60-120-15-1.5 (n-p-k-sulfur-zinc) was applied. Left as missing.
mask = temp.Fertilizer_info == '150 lbs/acre in UAN form applied with chemical ahead of tillage (N) 18-60-120-15-1.5 (n-p-k-sulfur-zinc) (P) 120lbs, fall applied (K)  [Uan, MESZ, and Potash]'
temp = temp.loc[~mask, :]
temp = temp.merge(pd.DataFrame(
{'Experiment_Code': ['IAI4', 'IAI4', 'IAI4', 'IAI4', 'IAI4', 'IAI4'],
'Fertilizer_Application_Datetime_1': ['11/20/14', '5/3/15', '5/3/15', '5/3/15', '5/3/15', '5/3/15'],
'Fertilizer_info': ['120 (K)', '150 (N)', '9999 (P)', '9999 (K)', '9999 (S)', '9999 (Zn)'] 
}
), how = 'outer')

mask = temp.Fertilizer_info == '167 (N) 80 (P) 60 (K)  [only total given by producer;  fall application of 17-80-60;  150 lbs. N in spring]'
temp = temp.loc[~mask, :]
temp = temp.merge(pd.DataFrame(
{'Experiment_Code': ['IAH3', 'IAH3', 'IAH3', 'IAH3'],
'Fertilizer_Application_Datetime_1': ['11/20/14', '11/20/14', '11/20/14', '5/3/15'],
'Fertilizer_info': ['17 (N)', '80 (P)', '60 (K)', '150 (N)'] 
}
), how = 'outer')     


temp = sanitize_col(
    df = temp, 
    col = 'Fertilizer_info', 
    simple_renames= {
    '166 (N) 30 (P) 40 (K)  [first app. 11-52-9 map (6 lbs N, 30 lbs. P) + 0-0-60 granular potash, spring app of 150 lbs. N as NH3]': '150 (N)',
    '200 (N) 0 (P) 0 (K)  [granular ammonium sulfate]': '200 (N)',
    '118 (N) 0 (P) 0 (K)  [UAN; 32-0-0]': '118 (N)',
    '180 (N) 0 (P) 0 (K)  [0.28]': '180 (N)',        
    '180 (N) 0 (P) 0 (K)  [NH3 + N-serve]': '180 (N)',
    '100 (N) 0 (P) 0 (K)  [UAN (28-0-0)]': '100 (N)',
    '0 (N) 0 (P) 0 (K)  [0]': '0 (N)',
    '80 (N) 0 (P) 0 (K)  [28% UAN]': '80 (N)',
    '160 (N) 0 (P) 0 (K)  [Anhydrous Ammonia]': '160 (N)',
    '100 (N) 0 (P) 0 (K)  [Ammonium Nitrate 34-0-0]': '100 (N)',
    '120 (N) 0 (P) 0 (K)  [0]': '120 (N)',
    '200 (N) 0 (P) 0 (K)  [Granular Urea]': '200 (N)',
    '100 (N) 0 (P) 0 (K)  [0]': '100 (N)'
    }, 
    split_renames= {
    '234 (N) 98 (P) 51 (K)': [
        '234 (N)', 
        '98 (P)', 
        '51 (K)'],
    '180 (N) 0 (P) 90 (K)  [10-0-30 24S]': [
        '180 (N)',                           
        '90 (K)',                       
        '9999 (S)'],
    '86 (N) 19 (P) 0 (K)  [32-0-0, 10-34-0]': [
        '86 (N)', 
        '19 (P)'],
    '196 (N) 92 (P) 120 (K)  [dry fertilizer + 160# Ammonia]': [
        '196 (N)', 
        '92 (P)', 
        '120 (K)'],
    '160 (N) 48 (P) 24 (K)  [Starter]': [
        '160 (N)', 
        '48 (P)', 
        '24 (K)'],          
    '155 (N) 50 (P) 100 (K)  [UAN 28% Dry PK]': [
        '155 (N)', 
        '50 (P)', 
        '100 (K)'],        
    '201 (N) 76 (P) 76 (K)  [19-19-19 without zinc & UAN 30% ]': [
        '201 (N)', 
         '76 (P)', 
         '76 (K)'],
    '138 (N) 46 (P) 62 (K)  [Diammonium phosphate, KCl, Super U (Urea)]': [
        '138 (N)', 
        '46 (P)', 
        '62 (K)'],
    '175 (N) 200 (P) 240 (K)  [granular 8-20-30, popup 10-34-0, liquid nitrogen 28-0-0-0.5]': [
        '175 (N)',  
        '200 (P)',  
        '240 (K)'],
    '102 (N) 40 (P) 40 (K)  [NPK and 30% UAN]': [
        '102 (N)', 
        '40 (P)', 
        '40 (K)'],
    '220 (N) 193 (P) 0 (K)  [11-37-0-5 zn, 32-0-0 UAN]': [
        '220 (N)', 
        '193 (P)', 
        '26.08108108108108 (Zn)'],
    '235 (N) 156 (P) 156 (K)  [10-20-20 & 19-19-19 with zink and UAN 30%]': [
        '235 (N)', 
        '156 (P)', 
        '156 (K)', 
        '9999 (Zn)'],          
    '275 (N) 200 (P) 240 (K)  [granular 8-20-30, popup 10-34-0, liquid nitrogen 28-0-0-0.5]': [
        '275 (N)', 
        '200 (P)',
        '240 (K)'],
    '220 (N) 193 (P) 0 (K)  [11-37-0-5 zn, 32-0-0]': [
        '220 (N)', 
        '193 (P)', 
        '26.08108108108108 (Zn)'], # solved for total Zn based on provided values
    '210.8 (N) 47.4 (P) 142.2 (K)  [10-10-30, 24S]': [
        '210.8 (N)', 
        '47.4 (P)', 
        '142.2 (K)'],
    '250 (N) 0 (P) 120 (K)  [300 lb/A 7-0-40 pre-plant 12.5gal/A 10-20-0-1 at planting 65gal/A UAN sidedress]': [
        '250 (N)',
        '120 (K)']
        
    })

In [ ]:
temp.loc[:, 'Product'] = np.nan
temp.loc[:, 'Amount_Per_Acre'] = np.nan

# assume each string is formated as 'val (key)'. `sanitize_col` should be used to enforce this.
for e in ['19 (P)', '48 (P)', '51 (K)', '235 (N)', '80 (P)', '86 (N)', '9999 (S)', '102 (N)', '100 (K)', '156 (P)', '92 (P)', '220 (N)', '193 (P)', '9999 (K)', '76 (K)', '0 (N)', '180 (N)', '9999 (Zn)', '90 (K)', '24 (K)', '9999 (P)', '155 (N)', '250 (N)', '100 (N)', '62 (K)', '40 (K)', '234 (N)', '26.08108108108108 (Zn)', '156 (K)', '240 (K)', '98 (P)', '120 (N)', '196 (N)', '175 (N)', '50 (P)', '200 (P)', '46 (P)', '120 (K)', '80 (N)', '80 (K)', '76 (P)', '200 (N)', '150 (N)', '160 (N)', '138 (N)', '40 (P)', '118 (N)', '201 (N)', '142.2 (K)', '275 (N)', '47.4 (P)', '60 (K)', '210.8 (N)', '17 (N)']:
    val = re.findall('^\d+[.]*\d*', e)[0]
    key = re.findall('\(.+\)',      e)[0].replace('(', '').replace(')', '')
    
    mask = (temp['Fertilizer_info'] == e)
    temp.loc[mask, 'Product'] = key
    temp.loc[mask, 'Amount_Per_Acre'] = val

In [ ]:
# Convert to long format, spread total values across n applications
temp = pd.melt(temp, id_vars=['Experiment_Code', 'Fertilizer_info', 'Imputation_Notes', 'Product', 'Amount_Per_Acre'])

temp = temp.rename(columns={e: 'Amount_Per_Acre', 'value': 'Date_Datetime'})
temp = temp.loc[pd.Series.notna(temp.Date_Datetime), :]
temp = temp.loc[temp.Amount_Per_Acre.notna(), :]
# now that we have removed the nas that are likely 0 we can set the unknowns from 9999 to np.nan
mask = (temp.Amount_Per_Acre == 9999)
temp.loc[mask, 'Amount_Per_Acre'] = np.nan

tally = temp.assign(n = 1).groupby('Experiment_Code').agg(n = ('n', np.sum)).reset_index()
temp = temp.merge(tally)
safe_create_col(temp, 'Unit')
temp.loc[:, 'Unit'] = 'lbs/Acre'

temp.Amount_Per_Acre = temp.Amount_Per_Acre.astype('float64')
temp.Amount_Per_Acre = temp.Amount_Per_Acre/temp.n
temp = temp.drop(columns= ['variable', 'n'])

##  Move management columns from meta to mgmt

In [ ]:
mgmt = mgmt.merge(temp, how = 'outer'
          ).merge(meta.loc[:, [
           'Experiment_Code',
           'Pre_Plant_Herbicide',
           'Post_Plant_Herbicide',
           'Insecticide']].drop_duplicates(), how = 'outer')

In [ ]:
meta = meta.drop(columns=[
    'Pre_Plant_Herbicide',
    'Post_Plant_Herbicide',
    'Insecticide',
    'N_Unit_lbs_per_A',
    'P_Unit_lbs_per_A',
    'K_Unit_lbs_per_A',
    'Fertilizer_Product',
    'Fertilizer_Application_Datetime_1',
    'Fertilizer_Application_Datetime_2',
    'Fertilizer_Application_Datetime_3',
    'Fertilizer_Application_Datetime_4',
    'Fertilizer_Application_Datetime_5',
    'Fertilizer_Application_Datetime_6',
    'Fertilizer_Application_Datetime_7',
    'Fertilizer_Application_Datetime_8'])

In [ ]:
# separate static and dynamic values
sval = phno.merge(soil, how = 'outer')
sval = sval.merge(meta.drop(columns='Test_Weight_Unit_lbs'), how = 'outer')

In [ ]:
# these tables are different enought we'll keep them separate
# mgmt
# unfortunately we need multiples because at least one field treats different passes differently
mgmt = phno.loc[:, ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'phno']
               ].drop_duplicates().merge(mgmt, how = 'outer')

In [ ]:
mgmt = mgmt.loc[mgmt.mgmt.notna(), :].drop(columns = 'phno')

In [ ]:
# Set each id col to a string
for i in ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot']:
    sval[i] = sval[i].astype('string')
    mgmt[i]  =  mgmt[i].astype('string')
    
    if i not in ['Range', 'Pass', 'Plot']:
        wthr[i]  =  wthr[i].astype('string')

# Sanitize Non-ID columns



## Sanitization functions

The pattern to use is:
 1. Alter the dataframe
 1. Test the dataframe against expectations
 
The main tasks that need to be completed are:
 1. Identify values that can't be converted to the expected data type. The "find_unconvertable_" family of functions should be used. 
     1. `find_unconvertable_datetimes`
     
 1. For simple renaming (e.g. misspellings) or splitting non-tidy data into two rows ("entry1-entry2" -> "entry1", "entry2") use `sanitize_col` 
 1. Move values that are ambigous but pertain to data imputation to "Imputation_Notes" using `relocate_to_Imputation_Notes`
 1. If new columns need to be added (e.g. mgmt.Ingredient for parsed components of Product (e.g. elements) ) this should be accomplished with `safe_create_col`.
 1. Any one off changes should be accomplised manually. 
 1. Confirm columns match the expected types with `check_df_dtype_expectations`, and report mismatches. 


These steps should be completed for each dataframe in turn to minimize the cognitive load of the reader. 

## Sanitization: Column data type expectations
Note: to handle missing values some columns that would otherwise be ints are floats

In [ ]:
sval_col_dtypes = mk_dtype_dict(name = 'sval')
wthr_col_dtypes = mk_dtype_dict(name = 'wthr')
mgmt_col_dtypes = mk_dtype_dict(name = 'mgmt')

,Year,Experiment_Code,Range,Pass,Plot,Irrigation_Applied,Weather_Station_Documents_Irrigation,Application,Product,Date_Datetime,Amount_Per_Acre,Unit,Nutrients_Applied,Management_Comments,mgmt,Fertilizer_info,Imputation_Notes,Pre_Plant_Herbicide,Post_Plant_Herbicide,Insecticide
0,2015.0,DEH1,2.0,1.0,1.0,Yes,Yes,irrigation,water,NaN,NaN,in/Acre,NaN,NaN,True,NaN,NaN,"Lexar 3qt/A combo of Metolachlor, Atrazine and...",NaN,Force 3G 5.5 lb/A at planting
1,2015.0,DEH1,2.0,1.0,1.0,Yes,Yes,irrigation,water,5/15/2015,0.5,in/Acre,NaN,NaN,True,NaN,NaN,"Lexar 3qt/A combo of Metolachlor, Atrazine and...",NaN,Force 3G 5.5 lb/A at planting
2,2015.0,DEH1,2.0,1.0,1.0,Yes,Yes,irrigation,water,5/28/2015,0.3,in/Acre,NaN,NaN,True,NaN,NaN,"Lexar 3qt/A combo of Metolachlor, Atrazine and...",NaN,Force 3G 5.5 lb/A at planting
3,2015.0,DEH1,2.0,1.0,1.0,Yes,Yes,irrigation,water,6/16/2015,0.8,in/Acre,NaN,NaN,True,NaN,NaN,"Lexar 3qt/A combo of Metolachlor, Atrazine and...",NaN,Force 3G 5.5 lb/A at planting
4,2015.0,DEH1,2.0,1.0,1.0,Yes,Yes,irrigation,water,7/1/2015,0.3,in/Acre,NaN,NaN,True,NaN,NaN,"Lexar 3qt/A combo of Metolachlor, Atrazine and...",NaN,Force 3G 5.5 lb/A at planting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59600,<NA>,SDI1,<NA>,<NA>,<NA>,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
59601,<NA>,TXI2,<NA>,<NA>,<NA>,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,"1.66 pts Dual; S-metolachlor, 1.5 lbs atrazine","Prowl H2O; pendamethaline, 1 lb Atrazine",NaN
59611,<NA>,TXI3,<NA>,<NA>,<NA>,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
59612,<NA>,WII1,<NA>,<NA>,<NA>,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,"Tank Mix (Callisto, AI Mesotrione) (Dual 2 Mag...",Counter Force 3G (AI Tefluthrin)


# Sanitization: Alter entries

## Static values (within season)

### Datetime containing columns

In [ ]:
sval.loc[sval.Recieved_Date_Unit_Datetime == 'See Soil Sample', 'Recieved_Date_Unit_Datetime'] = np.nan

In [ ]:
# convert the date cols into datetime. Lean on pd.to_datetime() to infer the format, assume that each site uses the same format.

for e in ['Planted_Unit_Datetime', 
    'Harvested_Unit_Datetime', 
    'Anthesis_Unit_Datetime', 
    'Silking_Unit_Datetime', 
    'Recieved_Date_Unit_Datetime', 
#     'Processed_Date_Unit_Datetime', 
    'Weather_Station_Placed_Unit_Datetime', 
    'Weather_Station_Removed_Unit_Datetime'
    ]:
# find_unconvertable_datetimes(df_col=sval[e], pattern='%Y-%m-%d %H:%M', index=False)

    sval['Datetime_Temp'] = pd.to_datetime(np.nan)
    print(e)
    for code in list(sval.Experiment_Code.drop_duplicates()):

    # code = list(sval.Experiment_Code.drop_duplicates())[0]
        sval.loc[sval.Experiment_Code == code, 'Datetime_Temp'
                 ] = pd.to_datetime(sval.loc[sval.Experiment_Code == code, e])

    sval.loc[:, e] = sval.loc[:, 'Datetime_Temp'] 

sval = sval.drop(columns = 'Datetime_Temp')

Planted_Unit_Datetime
Harvested_Unit_Datetime
Anthesis_Unit_Datetime
Silking_Unit_Datetime
Recieved_Date_Unit_Datetime
Weather_Station_Placed_Unit_Datetime
Weather_Station_Removed_Unit_Datetime


In [ ]:
# to bool
sval = sanitize_col(
    df = sval, 
    col = 'Discarded', 
    simple_renames= {
        'Yes':'True',
        'yes':'True'}, 
    split_renames= {})

# set missing to false
sval.loc[sval.Discarded.isna(), 'Discarded'] = 'False'
sval.Discarded = sval.Discarded.map({'True': True, 'False': False})

### Simple Columns

In [ ]:
# # to float
# # sval.Pounds_Needed_Soil_Moisture.astype(float)
sval = sval.drop(columns=['Drop_Record_Index', 'Additional_Metics'])

In [ ]:
# to bool
sval['phno'] = sval['phno'].astype('bool')
sval['soil'] = sval['soil'].astype('bool')
sval['meta'] = sval['meta'].astype('bool')

# to string
sval = cols_astype_string(
    df = sval, 
    col_list = [key for key in sval_col_dtypes.keys() if sval_col_dtypes[key] == 'string'])

sval.Year = year_string
sval.Year = sval.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = sval, dtype_dct = sval_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])

72/72 Columns pass.


## Weather

### Datetime

In [ ]:
# ... or we use the fields in the df to make a consistent format
wthr = cols_astype_string(
    df = wthr, 
    col_list = ['Year', 'Month', 'Day', 'Time'])

wthr['Datetime_Temp'] = wthr['Year']+'-'+wthr['Month']+'-'+wthr['Day']+' '+wthr['Time']

wthr['Datetime'] = pd.to_datetime(pd.Series(wthr.Datetime_Temp))
wthr = wthr.drop(columns= 'Datetime_Temp')

### Photoperiod

In [ ]:
mask = wthr.Photoperiod_Unit_Hours.notna()
temp = wthr.Photoperiod_Unit_Hours.str.split(":", expand=True)
wthr.Photoperiod_Unit_Hours = temp.loc[:, 0].astype(float) + (temp.loc[:, 1].astype(float)/60)

### Data_Cleaned

In [ ]:
# to bool
wthr = sanitize_col(
    df = wthr, 
    col = 'Data_Cleaned', 
    simple_renames= {
        'Yes':'True',
        'No':'False'}, 
    split_renames= {})

# set missing to false
wthr.loc[wthr.Data_Cleaned.isna(), 'Data_Cleaned'] = 'False'
wthr.Data_Cleaned = wthr.Data_Cleaned.map({'True': True, 'False': False})

### Simple Columns 

In [ ]:
wthr = wthr.drop(columns=['Drop_Record_Index',
                         'CO2_Unit_ppm' # 0 non null
                         ])

In [ ]:
# to string
wthr = cols_astype_string(
    df = wthr, 
    col_list = [key for key in wthr_col_dtypes.keys() if wthr_col_dtypes[key] == 'string'])

wthr.Year = year_string
wthr.Year = wthr.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = wthr, dtype_dct = wthr_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])


28/28 Columns pass.


## Management

### Date_Datetime

In [ ]:
# convert types
err_list = find_unconvertable_datetimes(df_col=mgmt.Date_Datetime, pattern='%m/%d/%Y', index=False)
if err_list != []:
    print(err_list)
else:
    mgmt.Date_Datetime = pd.to_datetime(pd.Series(mgmt.Date_Datetime), format = '%m/%d/%Y', errors='coerce')

### Amount_Per_Acre

In [ ]:
# effectively done above.

In [ ]:
# convert types
err_list = find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = False)
if err_list != []:
    print(err_list)
else:
    mgmt.Amount_Per_Acre = pd.to_numeric(mgmt.Amount_Per_Acre, errors='coerce')

### Ingredient
This is to be the cleaned up version of the "Product" column

In [ ]:
# list(mgmt.loc[:, 'Ingredient'].drop_duplicates())

### Simple Columns

In [ ]:
# to bool
mgmt['mgmt'] = mgmt['mgmt'].astype('bool')

# to string
for e in [ee for ee in ['Irrigation_Applied', 'Weather_Station_Documents_Irrigation', 'Application', 'Product', 'Unit', 'Nutrients_Applied', 'Management_Comments', 'Fertilizer_info', 'Imputation_Notes', 'Pre_Plant_Herbicide', 'Post_Plant_Herbicide', 'Insecticide'] if ee in mgmt.columns]:
    mgmt[e] = mgmt[e].astype('string')
    

mgmt.Year = year_string
mgmt.Year = mgmt.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = mgmt, dtype_dct = mgmt_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])

20/20 Columns pass.


# Publish



In [ ]:
write_out_pkl(obj = sval, path = './data/interim/'+year_string+'sval.pickle')
write_out_pkl(obj = wthr, path = './data/interim/'+year_string+'wthr.pickle')
write_out_pkl(obj = mgmt, path = './data/interim/'+year_string+'mgmt.pickle')